In [1]:
import json
import numpy as np
import pandas as pd

import torch
from torch.nn.functional import cosine_similarity
from sentence_transformers import SentenceTransformer, util

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
#load and clean the tuned predictions
tuned_responses = json.load(open("../Data/gemini_tuned_response.json",'r'))
tuned_responses = [response.strip() for response in tuned_responses]

#load and clean the base responses
base_responses = json.load(open("../Data/gemini_base_response.json",'r'))
base_responses = [base_responses[i].strip() for i in range(len(base_responses))]

#load and clean the microsoft titles
microsoft_titles = json.load(open("../Data/microsoft_titles.json",'r'))
microsoft_titles = [microsoft_titles[i].strip() for i in range(len(microsoft_titles))]

In [4]:
#convert suggested titles to embeddings using Sentence Transformer (model: all-MiniLM-L6-v2)
base_embeddings = [model.encode(base_responses[i], convert_to_tensor=True) for i in range(len(base_responses))]
tuned_embeddings = [model.encode(tuned_responses[i], convert_to_tensor=True) for i in range(len(tuned_responses))]
microsoft_embeddings = [model.encode(microsoft_titles[i], convert_to_tensor=True) for i in range(len(microsoft_titles))]

In [5]:
#apply mean pooling embeddings and arrange the dimensions
base_stacked_tensors = torch.stack(base_embeddings)
base_mean_tensor = torch.mean(base_stacked_tensors,dim = 0).unsqueeze(0)

tuned_stacked_tensors = torch.stack(tuned_embeddings)
tuned_mean_tensor = torch.mean(tuned_stacked_tensors,dim = 0).unsqueeze(0)

microsoft_stacked_tensors = torch.stack(microsoft_embeddings)
microsoft_mean_tensor = torch.mean(microsoft_stacked_tensors,dim = 0).unsqueeze(0)

In [6]:
performance = pd.DataFrame(
    {
        "Similarity": [
            float(cosine_similarity(microsoft_mean_tensor, microsoft_mean_tensor)[0]),
            float(cosine_similarity(tuned_mean_tensor, microsoft_mean_tensor)[0]),
            float(cosine_similarity(microsoft_mean_tensor, base_mean_tensor)[0]),
        ]
    },
    index=["Phi vs Phi", "Phi vs Tuned Gemini", "Phi vs Gemini"],
)

print(performance)

                     Similarity
Phi vs Phi             1.000000
Phi vs Tuned Gemini    0.882114
Phi vs Gemini          0.767320
